# Pathlib Challenges

For these challenges we will be getting familiar with the `pathlib` library.

This topic was inspired by our own Organizer, Chris May. He wrote an article on [getting started with pathlib](https://everydaysuperpowers.dev/articles/stop-working-so-hard-paths-get-started-pathlib/) which also prompted him to write a [field guide](https://everydaysuperpowers.dev/documents/3/ES-Getting_Started_with_Pathlib.pdf) and [cheat sheet](https://everydaysuperpowers.dev/documents/2/pathlib_cheat_sheet-V1_200703.pdf). You can see his other articles and resources at https://everydaysuperpowers.dev. Feel free to look at the resources and the `pathlib` [documentation](https://docs.python.org/3/library/pathlib.html) as they serve as the basis for the exercises below.

There are two kinds of Paths. The documentation states, "*Path classes are divided between pure paths, which provide purely computational operations without I/O, and concrete paths, which inherit from pure paths but also provide I/O operations.*" In other words,

* **`PurePath`**: Performs path operations witout caring about what might actually be on the disk.
* **`Path`**: Allows you to interact with files.

Both `PurePath` and `Path` can be either Windows or Posix paths. Chances are that you will not need to worry about the operating system specific choice since `pathlib` will take care of all the specifics for you!

One common shortcut to get the path of the current file is with the `__file__` variable. Due to how Jupyter Notebooks work, the `__file__` variable is not available, so it is mimmicked in the imports block. If you're not familiar with `__file__`, I suggest you look at [`__name__`](https://docs.python.org/3/library/__main__.html) and how python uses [dunder](https://dbader.org/blog/python-dunder-methods) attributes and methods.

In [1]:
import json
import os
import string
from pathlib import Path
from exercise import setup, cleanup

setup()
__file__ = os.path.join(os.getcwd(), 'Challenge.ipynb')
__file__

100%|██████████| 200/200 [00:48<00:00,  4.10it/s]


'/home/cohan/Documents/projects/coding-night/pathlib/Challenge.ipynb'

# Where am I?

The Zen of Python states

    There should be one-- and preferably only one --obvious way to do it.
    Although that way may not be obvious at first unless you're Dutch.
    
So naturally it would follow that there are many ways to access the current directory as a place to start. There are 11 different methods shown below for gettinging the current directory. Take some time and look over how each line works. Some things to note:

* `os.getcwd()` is shown, but only included for comparison purposes.
* In all major operating systems `.` refers to the current directory and `..` referrs to the directory above.
* Note the difference between lines with and without `.resolve()`
* Note how `__file__` requires the `parent` attribute.
* Note the difference between `.parent` and `.parents`.

While these are all interchangable here, they are not always interchangeable. Some will refer to the working directory while others refer to the location of the script. Be careful with which one you choose. 

In [2]:
[
    os.getcwd(),
    Path(os.getcwd()),
    Path.cwd(),
    Path(),
    Path().resolve(),
    Path('.'),
    Path('.').resolve(),
    Path(__file__) / '..',
    (Path(__file__) / '..').resolve(),
    Path(__file__).parent,
    Path(__file__).parents[0],
]

['/home/cohan/Documents/projects/coding-night/pathlib',
 PosixPath('/home/cohan/Documents/projects/coding-night/pathlib'),
 PosixPath('/home/cohan/Documents/projects/coding-night/pathlib'),
 PosixPath('.'),
 PosixPath('/home/cohan/Documents/projects/coding-night/pathlib'),
 PosixPath('.'),
 PosixPath('/home/cohan/Documents/projects/coding-night/pathlib'),
 PosixPath('/home/cohan/Documents/projects/coding-night/pathlib/Challenge.ipynb/..'),
 PosixPath('/home/cohan/Documents/projects/coding-night/pathlib'),
 PosixPath('/home/cohan/Documents/projects/coding-night/pathlib'),
 PosixPath('/home/cohan/Documents/projects/coding-night/pathlib')]

Okay, now that you've looked at the different ways of accessing the current directory, use one of them to set a variable `base_dir` to the base of the repo (one parent up)

In [3]:
base_dir = Path().resolve().parent

How many directories are we away from the file system root? What are they?

In [4]:
[*base_dir.resolve().parents]

[PosixPath('/home/cohan/Documents/projects'),
 PosixPath('/home/cohan/Documents'),
 PosixPath('/home/cohan'),
 PosixPath('/home'),
 PosixPath('/')]

# Who am I?

Often, you might want to access the home directory of the user. Create a variable `home_dir` that refers to the current user's home directory (In windows, this is `C:\Users\<user>`, in Linux it's usually `/home/<user>`). Does the Path look right? If you launched this notebook by clicking the Binder link in the repo, [this](https://github.com/jupyter/docker-stacks/issues/358#issuecomment-288844834) might answer a question you're now asking.

**You should not be hard coding the file path.**

In [5]:
home_dir = Path.home()
home_dir

PosixPath('/home/cohan')

# What Have We Got Here?

Now that you know where you are, it's time to see what is there with you.

List all the items in the home directory.

In [6]:
[f for f in home_dir.iterdir()]

[PosixPath('/home/cohan/.ipvanish'),
 PosixPath('/home/cohan/.PyCharm2019.2'),
 PosixPath('/home/cohan/.ipynb_checkpoints'),
 PosixPath('/home/cohan/.wget-hsts'),
 PosixPath('/home/cohan/.inputrc'),
 PosixPath('/home/cohan/.vscode'),
 PosixPath('/home/cohan/Downloads'),
 PosixPath('/home/cohan/Documents'),
 PosixPath('/home/cohan/.thumbnails'),
 PosixPath('/home/cohan/.local'),
 PosixPath('/home/cohan/.gnupg'),
 PosixPath('/home/cohan/.ipython'),
 PosixPath('/home/cohan/dwhelper'),
 PosixPath('/home/cohan/.dropbox'),
 PosixPath('/home/cohan/.zoom'),
 PosixPath('/home/cohan/.ICEauthority'),
 PosixPath('/home/cohan/.java'),
 PosixPath('/home/cohan/.fs'),
 PosixPath('/home/cohan/.netrc'),
 PosixPath('/home/cohan/.docker'),
 PosixPath('/home/cohan/snap'),
 PosixPath('/home/cohan/.gitconfig'),
 PosixPath('/home/cohan/.steampath'),
 PosixPath('/home/cohan/.steampid'),
 PosixPath('/home/cohan/.profile'),
 PosixPath('/home/cohan/Dropbox'),
 PosixPath('/home/cohan/.steam'),
 PosixPath('/home/co

Only list items in the home directory that are files.

In [7]:
[f for f in home_dir.iterdir() if f.is_file()]

[PosixPath('/home/cohan/.wget-hsts'),
 PosixPath('/home/cohan/.inputrc'),
 PosixPath('/home/cohan/.ICEauthority'),
 PosixPath('/home/cohan/.fs'),
 PosixPath('/home/cohan/.netrc'),
 PosixPath('/home/cohan/.gitconfig'),
 PosixPath('/home/cohan/.steampid'),
 PosixPath('/home/cohan/.profile'),
 PosixPath('/home/cohan/.sudo_as_admin_successful'),
 PosixPath('/home/cohan/.bash_aliases'),
 PosixPath('/home/cohan/.python_history'),
 PosixPath('/home/cohan/.pulse-cookie'),
 PosixPath('/home/cohan/.selected_editor'),
 PosixPath('/home/cohan/.bash_history'),
 PosixPath('/home/cohan/.bashrc'),
 PosixPath('/home/cohan/.bash_logout')]

Only list items in the home directory that are directories.

In [8]:
[d for d in home_dir.iterdir() if d.is_dir()]

[PosixPath('/home/cohan/.ipvanish'),
 PosixPath('/home/cohan/.PyCharm2019.2'),
 PosixPath('/home/cohan/.ipynb_checkpoints'),
 PosixPath('/home/cohan/.vscode'),
 PosixPath('/home/cohan/Downloads'),
 PosixPath('/home/cohan/Documents'),
 PosixPath('/home/cohan/.thumbnails'),
 PosixPath('/home/cohan/.local'),
 PosixPath('/home/cohan/.gnupg'),
 PosixPath('/home/cohan/.ipython'),
 PosixPath('/home/cohan/dwhelper'),
 PosixPath('/home/cohan/.dropbox'),
 PosixPath('/home/cohan/.zoom'),
 PosixPath('/home/cohan/.java'),
 PosixPath('/home/cohan/.docker'),
 PosixPath('/home/cohan/snap'),
 PosixPath('/home/cohan/Dropbox'),
 PosixPath('/home/cohan/.steam'),
 PosixPath('/home/cohan/.virtualenvs'),
 PosixPath('/home/cohan/.dropbox-dist'),
 PosixPath('/home/cohan/Pictures'),
 PosixPath('/home/cohan/Videos'),
 PosixPath('/home/cohan/Music'),
 PosixPath('/home/cohan/.mono'),
 PosixPath('/home/cohan/.cache'),
 PosixPath('/home/cohan/Templates'),
 PosixPath('/home/cohan/.PyCharm2019.3'),
 PosixPath('/home/c

Find all the notebooks under `base_dir` (`.ipynb`)

In [9]:
[f for f in base_dir.rglob('*.ipynb')]

[PosixPath('/home/cohan/Documents/projects/coding-night/pathlib/Answers.ipynb'),
 PosixPath('/home/cohan/Documents/projects/coding-night/pathlib/.ipynb_checkpoints/Answers-checkpoint.ipynb'),
 PosixPath('/home/cohan/Documents/projects/coding-night/venv/lib/python3.7/site-packages/nbformat/tests/test4plus.ipynb'),
 PosixPath('/home/cohan/Documents/projects/coding-night/venv/lib/python3.7/site-packages/nbformat/tests/test4.ipynb'),
 PosixPath('/home/cohan/Documents/projects/coding-night/venv/lib/python3.7/site-packages/nbformat/tests/test4custom.ipynb'),
 PosixPath('/home/cohan/Documents/projects/coding-night/venv/lib/python3.7/site-packages/nbformat/tests/test4docinfo.ipynb'),
 PosixPath('/home/cohan/Documents/projects/coding-night/venv/lib/python3.7/site-packages/nbformat/tests/invalid.ipynb'),
 PosixPath('/home/cohan/Documents/projects/coding-night/venv/lib/python3.7/site-packages/nbformat/tests/test2.ipynb'),
 PosixPath('/home/cohan/Documents/projects/coding-night/venv/lib/python3.7/

What versions of python are installed on this machine? (If you're running linux or mac, look for files containing `python` under `/usr/bin`. If using windows, skip this and the next challenge.)

In [10]:
[f for f in Path('/usr/bin').glob('python*')]

[PosixPath('/usr/bin/python3.6-config'),
 PosixPath('/usr/bin/python3m-config'),
 PosixPath('/usr/bin/python3m'),
 PosixPath('/usr/bin/python-config'),
 PosixPath('/usr/bin/python'),
 PosixPath('/usr/bin/python2-config'),
 PosixPath('/usr/bin/python3.8'),
 PosixPath('/usr/bin/python3.6m-config'),
 PosixPath('/usr/bin/python3.6m'),
 PosixPath('/usr/bin/python2.7'),
 PosixPath('/usr/bin/python3.7'),
 PosixPath('/usr/bin/python2.7-config'),
 PosixPath('/usr/bin/python3.7m'),
 PosixPath('/usr/bin/python3'),
 PosixPath('/usr/bin/python2'),
 PosixPath('/usr/bin/python3.6'),
 PosixPath('/usr/bin/python3-config')]

Which python files are just symbolic links to actual files?

In [11]:
[f for f in Path('/usr/bin').glob('python*') if f.is_symlink()]

[PosixPath('/usr/bin/python3.6-config'),
 PosixPath('/usr/bin/python3m-config'),
 PosixPath('/usr/bin/python3m'),
 PosixPath('/usr/bin/python-config'),
 PosixPath('/usr/bin/python'),
 PosixPath('/usr/bin/python2-config'),
 PosixPath('/usr/bin/python3.6m-config'),
 PosixPath('/usr/bin/python2.7-config'),
 PosixPath('/usr/bin/python3'),
 PosixPath('/usr/bin/python2'),
 PosixPath('/usr/bin/python3-config')]

Print the contents of `requirements.txt` under `base_dir`. Do this with one line.

In [12]:
print(base_dir.joinpath('requirements.txt').read_text())

beautifulsoup4>=4.9.1
jupyterlab>=2.1.4
nbstripout>=0.3.8
python-slugify>=4.0.0
requests>=2.23.0
scrapy>=2.1.0
selenium>=3.141.0
tabulate>=0.8.7
tqdm>=4.47.0


# File Properties

In [13]:
# year_ago = datetime.datetime.now() - datetime.timedelta(days=365)

# for f in Path.home().iterdir():
#     try:
#         mtime = datetime.datetime.fromtimestamp(f.stat().st_mtime)
#         if mtime < year_ago:
#             print(f, mtime)
#     except FileNotFoundError:
#         pass

# Rename Files

The `setup()` function at the top of the notebook downloaded images from the first 10 pages from http://books.toscrape.com/ and compiled the data into a json file.

The json file and images are located in a directory called `books`. List all the images in the directory. (They are all jpg)

In [14]:
[f for f in Path('books').rglob('*.jpg')]

[PosixPath('books/images/f34ffb24cc21c9f9f52dad4fd8f3ac21.jpg'),
 PosixPath('books/images/d61b12ef0e69d49e2645537dd6a8472a.jpg'),
 PosixPath('books/images/106e2fc7160712edf8e2ff996dc8cd6c.jpg'),
 PosixPath('books/images/ccbdae9e29b3594301528fa2c876ec29.jpg'),
 PosixPath('books/images/8a490347afdc10abe7c2099e466de32f.jpg'),
 PosixPath('books/images/261c4eaf957ae4aacf2229b482e76dbe.jpg'),
 PosixPath('books/images/974709d437b08e74649b5744471bf472.jpg'),
 PosixPath('books/images/553310a7162dfbc2c6d19a84da0df9e1.jpg'),
 PosixPath('books/images/0d1f3f934460f5a50aaa8c366641234c.jpg'),
 PosixPath('books/images/3e0b16851bec08b6cbf78d5f64af9114.jpg'),
 PosixPath('books/images/f85417465a73e33604624205ba8306cc.jpg'),
 PosixPath('books/images/6625e3bbb050de3e42a0c302c0d69f1f.jpg'),
 PosixPath('books/images/d3158e8d3546fb90cced3c1d44a92a34.jpg'),
 PosixPath('books/images/16d443437126bf6d536a89312c1995a5.jpg'),
 PosixPath('books/images/44ccc99c8f82c33d4f9d2afa4ef25787.jpg'),
 PosixPath('books/images/

Can you tell the titles of the different books? If not, load the json file and try to rename each file.

In [15]:
data = json.loads(list(Path('books').rglob('*.json'))[0].read_text())
mapping = {Path(book['img']).stem: book['name'] for book in data}

for img in Path('books').rglob('*.jpg'):
    new_name = img.with_name(mapping[img.stem] + img.suffix)
    img.rename(new_name)    

Now check to see that you can figure out which image is which. (copy your code from a few cells up.)

In [16]:
[f for f in Path('books').rglob('*.jpg')]

[PosixPath('books/images/Mrs. Houdini.jpg'),
 PosixPath('books/images/Modern Romance.jpg'),
 PosixPath("books/images/Scott Pilgrim's Precious Little Life (Scott Pilgrim #1).jpg"),
 PosixPath('books/images/Redeeming Love.jpg'),
 PosixPath('books/images/In Her Wake.jpg'),
 PosixPath('books/images/Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991.jpg'),
 PosixPath('books/images/Shobu Samurai, Project Aryoku (#3).jpg'),
 PosixPath('books/images/Reskilling America: Learning to Labor in the Twenty-First Century.jpg'),
 PosixPath("books/images/The Omnivore's Dilemma: A Natural History of Four Meals.jpg"),
 PosixPath('books/images/The Murder of Roger Ackroyd (Hercule Poirot #4).jpg'),
 PosixPath('books/images/Done Rubbed Out (Reightman & Bailey #1).jpg'),
 PosixPath('books/images/Slow States of Collapse: Poems.jpg'),
 PosixPath('books/images/Forever and Forever: The Courtship of Henry Longfellow and Fanny Appleton.jpg'),
 PosixPath('books/images/Crown of Midnig

# Let's Make Something

Under `base_dir`, Create a path to a file called `example.txt`.

In [17]:
file = base_dir / 'example.txt'

Check to see if the file exists

In [18]:
file.exists()

False

Write text to the file using one of the `pathlib` utility methods.

In [19]:
file.write_text('PyRVA')

5

See if the file exits now.

In [20]:
file.exists()

True

Get the text from the file using one of the `pathlib` utility methods.

In [21]:
file.read_text()

'PyRVA'

Under `base_dir`:

* Create a new directory called `new`. 
* In `new`, create 26 sub directories based on the letters of the alphabet (you can use `string.ascii_lowercase` to iterate if you'd like). 
* Under each letter directory, create 10 directories numbered `0` -> `9` (you can use `range(10)` to iterate if you'd like.) 
* In each numbered directory, create a file called `file.txt` and write `PyRVA is Awesome!` to each file.

You should only have one line in the code block that actually creates directories and one line in the code block that creates the file.

What happens if you run the code block twice?

In [22]:
for char in string.ascii_lowercase:
    for i in range(10):
        new = base_dir / 'new' / char / str(i)
        new.mkdir(parents=True)        
        (new / 'file.txt').write_text('PyRVA is Awesome!')

Find all the `file.txt` files under `base_dir` and check that the number matches what you expected (this can be done in one line). If not, you might need to debug something.

In [23]:
len([*base_dir.rglob('file.txt')])

261

In [24]:
[f for f in base_dir.rglob('file.txt') if 'new' not in str(f)]

[PosixPath('/home/cohan/Documents/projects/coding-night/pathlib/file.txt')]

Now delete the `new` folder.

In [25]:
def rmdir(top: Path):
    for file in top.iterdir():
        if file.is_file():
            file.unlink()
        elif file.is_dir():
            rmdir(file)
    top.rmdir()
            
rmdir(base_dir / 'new')

# Which Is Better?

For these exercises, see if you can figure out the pathlib replacement for the following code blocks. There is a quick reference table at the bottom of the `pathlib` documentation, but I encourage you to read through the documentation without looking at the table. Once you have verified the output to be the same (don't worry about the `PosixPath()` `__repr__` value, as long as the result is the same otherwise, you can use `str()` if you really care about it), use the [`%%timeit`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit) magic method to compare the speeds.

Once you have compared the different methods, think about which of the follow things are important to you:

* Speed of development (characters typed)
* Speed of maintenance (how easy is it to read)
* Speed of execution (how fast the code runs)

Getting the parent of the current folder.

In [26]:
os.path.dirname(os.path.dirname(os.path.abspath(__file__)))

'/home/cohan/Documents/projects/coding-night'

In [27]:
Path(__file__).parents[1]

PosixPath('/home/cohan/Documents/projects/coding-night')

Get the basename of the current file.

In [28]:
os.path.basename(__file__)

'Challenge.ipynb'

In [29]:
Path(__file__).name

'Challenge.ipynb'

Get the basename of the current file without the extension.

In [30]:
os.path.splitext(os.path.basename(__file__))[0]

'Challenge'

In [31]:
Path(__file__).stem

'Challenge'

Get the file extension of the current file.

In [32]:
os.path.splitext(__file__)[1]

'.ipynb'

In [33]:
Path(__file__).suffix

'.ipynb'

Create a `PurePath` with a different name in the same directory (file doesn't have to exist).

In [34]:
os.path.join(os.path.dirname(__file__), 'myfile.txt')

'/home/cohan/Documents/projects/coding-night/pathlib/myfile.txt'

In [35]:
Path(__file__).with_name('myfile.txt')

PosixPath('/home/cohan/Documents/projects/coding-night/pathlib/myfile.txt')

Create a `PurePath` of the same file with a different extension.

In [36]:
os.path.join(os.path.dirname(__file__), os.path.basename(os.path.splitext(__file__)[0]) + '.py')

'/home/cohan/Documents/projects/coding-night/pathlib/Challenge.py'

In [37]:
Path(__file__).with_suffix('.py')

PosixPath('/home/cohan/Documents/projects/coding-night/pathlib/Challenge.py')

Get the home directory of the current user.

In [38]:
os.path.expanduser('~')

'/home/cohan'

In [39]:
Path.home()

PosixPath('/home/cohan')

# Cleanup

Run the following line to remove files created by this exercise (assuming you followed the suggested file names).

In [40]:
cleanup()